In [1]:
from PIL import Image
import os
import numpy as np

def resize_and_pad(image_path, target_size=(64, 64), pad_color=(0, 0, 0, 0)):
    img = Image.open(image_path)

    if img.height < target_size[1]:
        new_img = Image.new('RGBA', target_size, pad_color)

        img_array = np.array(img.convert('RGBA'))
        new_img_array = np.array(new_img)
        alpha_channel = img_array[:, :, 3]

        x_start = (target_size[0] - img.width) // 2
        x_end = x_start + img.width
        y_start = 0
        y_end = img.height

        new_img_array[y_start:y_end, x_start:x_end, :3] = img_array[:, :, :3]
        new_img_array[y_start:y_end, x_start:x_end, 3] = alpha_channel

        new_img = Image.fromarray(new_img_array, 'RGBA')
        new_img.save(image_path, format='PNG')

# Проходим по всем изображениям в папке
folder_path = "./bigdata/Skins/"
for filename in os.listdir(folder_path):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        file_path = os.path.join(folder_path, filename)
        resize_and_pad(file_path)


In [2]:
from PIL import Image, ImageEnhance, ImageOps
from collections import Counter

color_classes = {
    (255, 0, 0): "Красный",
    (255, 165, 0): "Оранжевый",
    (255, 255, 0): "Жёлтый",
    (0, 255, 0): "Зелёный",
    (0, 0, 255): "Синий",
    (75, 0, 130): "Фиолетовый",
    (0, 0, 0): "Черный",
    (255, 255, 255): "Белый",
    (139, 69, 19): "Коричневый",
    (255, 182, 193): "Розовый",
    (128, 128, 128): "Серый",
}

In [3]:
def preprocess_image(image_path, posterize_bits=4, saturation_factor=2):
    img = Image.open(image_path)
    if img.mode != 'RGBA':
        img = img.convert('RGBA')
    
    # Увеличиваем насыщенность
    enhancer = ImageEnhance.Color(img)
    img = enhancer.enhance(saturation_factor)
    
    # Постеризация для уменьшения числа цветов
    img = posterize_image(img, posterize_bits)
    
    # Определение преобладающего цвета, пропуская прозрачные пиксели
    pixels = [pixel[:3] for pixel in img.getdata() if pixel[3] != 0]
    if pixels:
        dominant_color = find_closest_color(Counter(pixels).most_common(1)[0][0])
    else:
        dominant_color = (0, 0, 0)  # Если все пиксели прозрачные, установим черный цвет
    
    return img, dominant_color

def find_closest_color(target_color):
    # Находим ближайший цвет из color_classes
    closest_color = min(color_classes, key=lambda color: euclidean_distance(color, target_color))
    return closest_color

def euclidean_distance(color1, color2):
    # Евклидово расстояние между двумя цветами
    return ((color1[0] - color2[0])**2 + (color1[1] - color2[1])**2 + (color1[2] - color2[2])**2)**0.5

def posterize_image(img, bits):
    # Реализация альтернативной posterize для изображений RGBA
    data = img.getdata()
    posterized_data = [(int(r // (256 / bits)) * (256 // bits),
                        int(g // (256 / bits)) * (256 // bits),
                        int(b // (256 / bits)) * (256 // bits),
                        a) for r, g, b, a in data]
    posterized_img = Image.new('RGBA', img.size)
    posterized_img.putdata(posterized_data)
    return posterized_img

def resize_and_pad(image_path, target_size=(64, 64), pad_color=(0, 0, 0, 0)):
    img, dominant_color = preprocess_image(image_path)
    
    # Получение класса изображения по ближайшему цвету
    image_class = color_classes.get(dominant_color, "Неизвестный")
    
    img = ImageOps.fit(img, target_size, Image.LANCZOS, 0, (0.5, 0.5))
    img = ImageOps.expand(img, border=(0, 0, target_size[0] - img.width, target_size[1] - img.height), fill=pad_color)
    
    return img, image_class

In [35]:
# Пример использования:
image_path = "./tskins/10.jpg"
resized_image, image_class = resize_and_pad(image_path)
print(f"Класс изображения: {image_class}")
resized_image.show()

Класс изображения: Розовый


In [4]:
import os
from shutil import move

def process_and_move_images(image_folder):
    for filename in os.listdir(image_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(image_folder, filename)
            _, image_class = resize_and_pad(image_path)

            # Создаем подкаталог, если его еще нет
            class_folder = os.path.join(image_folder, image_class)
            os.makedirs(class_folder, exist_ok=True)

            # Перемещаем изображение в соответствующий подкаталог
            destination_path = os.path.join(class_folder, filename)
            move(image_path, destination_path)

# Пример использования:
image_folder = "./bigdata/Skins/"
process_and_move_images(image_folder)
